In [1]:
import sys, os, fnmatch, datetime, subprocess, imp
import numpy as np
from collections import defaultdict
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
fpath = os.getcwd()
DATADIR = os.path.join(fpath+'/data/')

In [5]:
match_dct = {
    'A':'T',
    'C':'G',
    'T':'A',
    'G':'C',
}

In [6]:
def reverse_complement(umi):
    if len(umi) == 0:
        return ''
    try:
        this = match_dct[umi[-1:]]
    except:
        print(umi)
        this = ''
    return this + reverse_complement(umi[:-1])

In [4]:
genos_df = pd.read_csv('exp_df_genos.csv')

FileNotFoundError: [Errno 2] File b'exp_df_genos.csv' does not exist: b'exp_df_genos.csv'

In [45]:
genos_df = genos_df.drop('Unnamed: 0', axis=1)

In [46]:
no_micro = genos_df.loc[lambda x: x['Genotype position'] != 'e']

In [ ]:
# removing microhomology --> predicted frequencies do not add up to 100%
no_micro.groupby('library_key')['Predicted frequency'].sum()

In [47]:
deletion = no_micro.loc[lambda x: x.Category == 'del']

In [48]:
insertion = no_micro.loc[lambda x: x.Category == 'ins']

In [49]:
lib = pd.read_csv('061318_exonskipping_library.csv')

In [50]:
lib_dict = pd.Series(lib.Sequence.values, index = lib['Identifier number']).to_dict()

In [51]:
target_dict = pd.Series(lib['Designed 61-bp target site (37i-24e, AG)'].values, index = lib['Identifier number']).to_dict()

In [52]:
cutsite_dict = pd.Series(lib['Cutsite within sequence'].values, index = lib['Identifier number']).to_dict()

Plan: take library sequence, and recreate deletions by using genotype position and deletion length --> go 3 bases to the right of genotype position, 20 bases to the left of the end of deletion. 
Insertions --> always added 1 base to the left. 
Handle deletions and insertions separately, then append dataframes together.

Deletions first!

In [53]:
deletion['lib'] = deletion.library_key.apply(lambda x: lib_dict[x])

/home/clara/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [54]:
deletion['RC'] = deletion.apply(lambda x: x.Genotype[:10] != x.lib[:10], axis=1)

/home/clara/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [55]:
deletion['target'] = deletion.library_key.apply(lambda x: target_dict[x])

/home/clara/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [56]:
deletion['cutsite'] = deletion.library_key.apply(lambda x: cutsite_dict[x])

/home/clara/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [ ]:
# deletion.lib = deletion.library_key.apply(lambda x: lib_dict[x] if x not in nonmatch_ids else reverse_complement(lib_dict[x]), axis=1)

confirm no more nonmatches left 

In [ ]:
# deletion['genotype_lib_match'] = deletion.apply(lambda x: x.Genotype[:10] == x.lib[:10], axis=1)

In [ ]:
# deletion.loc[deletion.genotype_lib_match == False]

In [38]:
def deletion_23(sequence, target, cutsite, pos, length):
    splice = sequence.index(target[:20]) + 37
    intron = sequence[:splice]
    exon = sequence[splice:]
    
    start = cutsite-1+int(pos)-int(length)
    end = start+int(length)
    
#     print(start)
#     print(end)
#     print(intron)
#     print(exon)

    if end < len(intron):
#         print('intron')
        intron_new = intron[:start] + intron[end:]
        exon_new = exon[:]
    elif start >= len(intron):
#         print('exon')
        exon_new = exon[:start-len(intron)+1] + exon[end-len(intron)+1:]
        intron_new = intron[:]
    else:
#         print('both')
        intron_new = intron[:start+1]
        exon_new = exon[end-len(intron)+1:]
#     print(intron_new)
#     print(exon_new)
    newstrand = intron_new+exon_new
#     print(newstrand)
#     print(start)
# should change to length of intron instead of start 
    index = newstrand.find('AG', start-5, start+5)
#     print(index)
    if index != -1:
        return newstrand[index-18:index+5]
    
    return intron_new[-20:]+exon_new[:3]

In [39]:
deletion_23('GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACTTGCCACCCATCGCGGCTCCCCTCCAGAAGGGGAACATTTACCTGGCCGACTACCGCATCATGGAGGGCATCCCCACCGTGGAGCTCAGCGGCCGGA',
           'GGGTGCCTACTTGCCACCCATCGCGGCTCCCCTCCAGAAGGGGAACATTTACCTGGCCGAC',
            70,
           2,
           11)

'CTTGCCACCCATCGCGGAAGGGG'

In [40]:
def deletion_23_rc(sequence, target, cutsite, pos, length):
    """
    pass in library sequence, not rc
    """
#     print(sequence.index(target[:30]))
    splice = sequence.index(target[:30]) + 37
    intron = sequence[:splice]
    exon = sequence[splice:]
    
#     print(intron)
#     print(exon)
    
    intron_rc = reverse_complement(intron)
    exon_rc = reverse_complement(exon)
    
    start = 142-cutsite+int(pos)-int(length)
    end = start+int(length)
  
    if end < len(exon_rc):
        # should it be len(exon_rc) - why didnt code break 
#         print('exon')
        exon_rc_new = exon_rc[:start] + exon_rc[end:]
        intron_rc_new = intron_rc[:]
    elif start >= len(exon_rc):
#         print('intron')
        intron_rc_new = intron_rc[:start-len(intron_rc)+2] + intron_rc[end-len(exon_rc):]
        exon_rc_new = exon_rc[:]
    else:
#         print('both')
        exon_rc_new = exon_rc[:start]
        intron_rc_new = intron_rc[end-len(exon_rc):]
#     print(exon_rc_new)
#     print(intron_rc_new)
    newstrand = reverse_complement(exon_rc_new+intron_rc_new)
#     print(newstrand)
#     print(start)
    index = newstrand.find('AG', len(exon_rc_new)-3, len(exon_rc_new)+3)
#     print(index)
    if index != -1:
        return newstrand[index-18:index+5]
    return reverse_complement(exon_rc_new[-3:]+intron_rc_new[:20])

In [43]:
deletion_23_rc('CCTCACAGCCCGGCCCTGACCTCCCTCCCCTCCCTCACAGCCCGGCCCTGACCTCCCTCCCTTCCCCCACAGGGTTCCACCTGTCCCACAAGGTCACCAGCGTGGTTCCCGAGAGCGCCCTGCTCATCGTGCTGGGCCTGGT',
              'CACAGCCCGGCCCTGACCTCCCTCCCTTCCCCCACAGGGTTCCACCTGTCCCACAAGGTCA',
              69, 23, 39)

'CCCCTCCCTCACAGCCCGGCCCA'

In [57]:
deletion['maxent_mer'] = deletion.apply(lambda x: deletion_23_rc(x.lib, x.target, x.cutsite, x['Genotype position'], x.Length) if x.RC  
                                        else deletion_23(x.lib, x.target, x.cutsite, x['Genotype position'], x.Length), axis=1)

/home/clara/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [58]:
deletion['deletion_length'] = deletion['maxent_mer'].apply(lambda x: len(x))

/home/clara/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [59]:
deletion.loc[(deletion.deletion_length < 23)&(deletion['Predicted frequency']>0.05)]

,Category,Genotype position,Inserted Bases,Length,Predicted frequency,Genotype,library_key,lib,RC,target,cutsite,maxent_mer,deletion_length


In [60]:
deletion.loc[(deletion.deletion_length > 23)]

,Category,Genotype position,Inserted Bases,Length,Predicted frequency,Genotype,library_key,lib,RC,target,cutsite,maxent_mer,deletion_length


Confirm all deletion lengths less than 23 have very low predicted frequency

find situations where the library does not match the genotype: reverse complement the genotype so that you can check the 23 mer is a substring

In [61]:
deletion['rc_genotype'] = deletion.apply(lambda x: reverse_complement(x.Genotype) if x.RC else x.Genotype, axis=1)

/home/clara/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Double check that the 23 mer is a substring of the deletion

In [62]:
deletion['in_genotype'] = deletion.apply(lambda x: True if x.maxent_mer in x.rc_genotype else False, axis=1)

/home/clara/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [87]:
'CGCAGGCCCTGCCCCCGGCCATG' in 'CCATGCTCCTCTCCAGGCTCAGCAGCAGGTACGTACCCAACCATGGGCTCGCAGGCCCTGCCCCCGCCCATGCAGACCCTCATCTTTTTCGACATGGAGGCCACTGGCTTGCCCTTCTCCCAGCCCAAGGTCACGGAGC'

False

In [63]:
deletion.loc[(deletion.in_genotype == False)&(deletion.deletion_length == 23)]

,Category,Genotype position,Inserted Bases,Length,Predicted frequency,Genotype,library_key,lib,RC,target,cutsite,maxent_mer,deletion_length,rc_genotype,in_genotype


In [64]:
del_df = deletion.loc[(deletion.in_genotype != False)&(deletion.deletion_length == 23)]

After filtering - similar match procedure on insertions, and generate 23 mers

In [65]:
insertion['lib'] = insertion.library_key.apply(lambda x: lib_dict[x])

/home/clara/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [66]:
insertion['RC'] = insertion.apply(lambda x: x.Genotype[:10] != x.lib[:10], axis=1)

/home/clara/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [67]:
insertion['target'] = insertion.library_key.apply(lambda x: target_dict[x])

/home/clara/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [68]:
insertion['cutsite'] = insertion.library_key.apply(lambda x: cutsite_dict[x])

/home/clara/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [92]:
def insert_23(sequence, target, cutsite, base):
    splice = sequence.index(target[:25]) + 37
    intron = sequence[:splice]
    exon = sequence[splice:]

    if cutsite < len(intron):
        intron_new = intron[:cutsite] + base+ intron[cutsite:]
        exon_new = exon[:]
    elif cutsite > len(intron):
        exon_new = exon[:cutsite-len(intron)] + base + exon[cutsite-len(intron):]
        intron_new = intron[:]
    else:
        intron_new = intron[:]
        exon_new = base + exon[:]
    
    newstrand = intron_new+exon_new
    
    index = newstrand.find('AG', splice-5, splice+5)
    if index != -1:
        return newstrand[index-18:index+5]
    
    return intron_new[-20:]+exon_new[:3]

In [93]:
insert_23('GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACTTGCCACCCATCGCGGCTCCCCTCCAGAAGGGGAACATTTACCTGGCCGACTACCGCATCATGGAGGGCATCCCCACCGTGGAGCTCAGCGGCCGGA',
         'GGGTGCCTACTTGCCACCCATCGCGGCTCCCCTCCAAGAAGGGGAACATTTACCTGGCCGAC',
         70, 
         'A')

'CATCGCGGCTCCCCTCCAAGAAG'

In [94]:
insert_23('CAGAAGGCAGGGAAGAGGGTTGGCATGGGGGGAGCATGCCCCAGCCTGACCTCTCCGACCCTCCCATGTTAGGTCCGGAAACAGCAGAAGGAGCACACTAATCTCATGGCAGAATATCGGAACAAGCAGCAGCAACAACAGC',
         'ATGCCCCAGCCTGACCTCTCCGACCCTCCCATGTTAGGTCCGGAAACAGCAGAAG',
         72,
         'A')

'CTCCGACCCTCCCATGTTAGAGT'

In [95]:
insert_23('GTCTGATCTCGGGAAACCTGGAGAAATTTATTTTCTGTACTCTAATGTTCTTTCATTTTGGTGACCATCAAGGTGCTGGGAGAGGAATTAGATGGCTGTAATTCAAAGTTAATGGAATTAGATGCAGCAGTACAGAAATTCT',
         'TGTACTCTAATGTTCTTTCATTTTGGTGACCATCAAGGTGCTGGGAGAGGAAT',
         73, 
         'A')

'TCATTTTGGTGACCATCAAGGAT'

In [107]:
insert_23('CCAAAGTGCTGGGATTACAGGCATGAGCCACTGTGCCCGACCCAGACTCTCACTTTCACCTGCGCCCCCCAGCGGCTCCTGCAGATGCAGGATGAAGGAAGAAGGCAGCCCAGACCTGGAGGCCGGGACCACATGGAGGCCC',
          'CCCGACCCAGACTCTCACTTTCACCTGCGCCCCCCAGCGGCTCCTGCAGATGCAGGATGAA',
         71, 
         'A')

'TTTCACCTGCGCCCCCCAAGCGG'

In [88]:
def insert_23_rc(sequence, target, cutsite, base):
    """
    pass in library sequence, not rc
    """
    splice = sequence.index(target[:30]) + 37
    intron = sequence[:splice]
    exon = sequence[splice:]
    
    intron_rc = reverse_complement(intron)
    exon_rc = reverse_complement(exon)
    
    add = 142-cutsite
    
    if add < len(exon_rc):
        exon_rc_new = exon_rc[:add] +base+ exon_rc[add:]
        intron_rc_new = intron_rc[:]
    elif add > len(exon_rc):
        intron_rc_new = intron_rc[:add-len(intron_rc)+2] + base + intron_rc[add-len(exon_rc):]
        exon_rc_new = exon_rc[:]
    else:
        intron_rc_new = base + intron_rc[:]
        exon_rc_new = exon_rc[:]
    
    newstrand = reverse_complement(exon_rc_new+intron_rc_new)

    index = newstrand.find('AG', splice-5, splice+5)

    if index != -1:
        return newstrand[index-18:index+5]
        
    return reverse_complement(exon_rc_new[-3:]+intron_rc_new[:20])

In [89]:
insert_23_rc('TTTCCTACATGACCAGTTGATTTTTGAGTCCTCAGAGACCTGAGCACAGTTCCTCTCTTTCAATCCCTCCAGGGTTCCAACAAAGTGAATGAAGCTCTTCTGCAAAGAATAAACAGTGCCAAAAAAATCCACTTGGTTCCAT',
            'AGACCTGAGCACAGTTCCTCTCTTTCAATCCCTCCAGGGTTCCAACAAAGT',
            71,
            'A')

'TCTCTTTCAATCCCTCCATGGGT'

In [90]:
insert_23_rc('CCTCACAGCCCGGCCCTGACCTCCCTCCCCTCCCTCACAGCCCGGCCCTGACCTCCCTCCCTTCCCCCACAGGGTTCCACCTGTCCCACAAGGTCACCAGCGTGGTTCCCGAGAGCGCCCTGCTCATCGTGCTGGGCCTGGT',
            'CACAGCCCGGCCCTGACCTCCCTCCCTTCCCCCACAGGGTTCCACCTGTCCCACAAGGTCA',
            69,
            'A')

'TCCCTCCCTTCCCCCATCAGGGT'

In [77]:
insert_23_rc('CCATGCTCCTCTCCAGGCTCAGCAGCAGGTACGTACCCAACCATGGGCTCGCAGGCCCTGCCCCCGGGGCCCATGCAGACCCTCATCTTTTTCGACATGGAGGCCACTGGCTTGCCCTTCTCCCAGCCCAAGGTCACGGAGC',
            'CCCAACCATGGGCTCGCAGGCCCTGCCCCCGGGGCCCATGCAGACCCTCATC',
            68,
            'A')

'GGCCCTGCCCCCGGGTGCCCATG'

In [108]:
insertion['maxent_mer'] = insertion.apply(lambda x: insert_23_rc(x.lib, x.target, x.cutsite, x['Inserted Bases']) if x.RC  
                                        else insert_23(x.lib, x.target, x.cutsite, x['Inserted Bases']), axis=1)

/home/clara/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [109]:
insertion['insert_length'] = insertion['maxent_mer'].apply(lambda x: len(x))

/home/clara/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [110]:
insertion.loc[(insertion.insert_length < 23)]

,Category,Genotype position,Inserted Bases,Length,Predicted frequency,Genotype,library_key,lib,RC,target,cutsite,maxent_mer,insert_length,rc_genotype,in_genotype


In [111]:
insertion.loc[(insertion.insert_length > 23)]

,Category,Genotype position,Inserted Bases,Length,Predicted frequency,Genotype,library_key,lib,RC,target,cutsite,maxent_mer,insert_length,rc_genotype,in_genotype


Confirm all deletion lengths less than 23 have very low predicted frequency

find situations where the library does not match the genotype: reverse complement the genotype so that you can check the 23 mer is a substring

In [112]:
insertion['rc_genotype'] = insertion.apply(lambda x: reverse_complement(x.Genotype) if x.RC else x.Genotype, axis=1)

/home/clara/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Double check that the 23 mer is a substring of the deletion

In [113]:
insertion['in_genotype'] = insertion.apply(lambda x: True if x.maxent_mer in x.rc_genotype else False, axis=1)

/home/clara/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [114]:
insertion.loc[(insertion.in_genotype == False)&(insertion.insert_length == 23)]

,Category,Genotype position,Inserted Bases,Length,Predicted frequency,Genotype,library_key,lib,RC,target,cutsite,maxent_mer,insert_length,rc_genotype,in_genotype


In [115]:
insert_df = insertion.loc[(insertion.in_genotype == True)]

In [116]:
complete_df = pd.concat([del_df, insert_df])

/home/clara/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [117]:
complete_df

,Category,Genotype,Genotype position,Inserted Bases,Length,Predicted frequency,RC,cutsite,deletion_length,in_genotype,insert_length,lib,library_key,maxent_mer,rc_genotype,target
0,del,GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACT...,2,NaN,11,1.319286,False,70,23.0,True,NaN,GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACT...,9251,CTTGCCACCCATCGCGGAAGGGG,GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACT...,GGGTGCCTACTTGCCACCCATCGCGGCTCCCCTCCAGAAGGGGAAC...
1,del,GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACT...,11,NaN,11,1.319286,False,70,23.0,True,NaN,GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACT...,9251,ACCCATCGCGGCTCCCCTCCATT,GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACT...,GGGTGCCTACTTGCCACCCATCGCGGCTCCCCTCCAGAAGGGGAAC...
2,del,GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACT...,2,NaN,12,1.027461,False,70,23.0,True,NaN,GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACT...,9251,ACTTGCCACCCATCGCGAAGGGG,GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACT...,GGGTGCCTACTTGCCACCCATCGCGGCTCCCCTCCAGAAGGGGAAC...
3,del,GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACT...,11,NaN,12,1.027461,False,70,23.0,True,NaN,GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACT...,9251,CACCCATCGCGGCTCCCCTCATT,GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACT...,GGGTGCCTACTTGCCACCCATCGCGGCTCCCCTCCAGAAGGGGAAC...
4,del,GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACT...,2,NaN,14,0.623186,False,70,23.0,True,NaN,GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACT...,9251,CTACTTGCCACCCATCGAAGGGG,GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACT...,GGGTGCCTACTTGCCACCCATCGCGGCTCCCCTCCAGAAGGGGAAC...
5,del,GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACT...,5,NaN,14,0.623186,False,70,23.0,True,NaN,GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACT...,9251,CTACTTGCCACCCATCGCGGGGG,GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACT...,GGGTGCCTACTTGCCACCCATCGCGGCTCCCCTCCAGAAGGGGAAC...
6,del,GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACT...,11,NaN,14,0.623186,False,70,23.0,True,NaN,GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACT...,9251,GCCACCCATCGCGGCTCCCCATT,GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACT...,GGGTGCCTACTTGCCACCCATCGCGGCTCCCCTCCAGAAGGGGAAC...
7,del,GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACT...,6,NaN,15,3.634922,False,70,23.0,True,NaN,GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACT...,9251,CTACTTGCCACCCATCGCGGGGA,GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACT...,GGGTGCCTACTTGCCACCCATCGCGGCTCCCCTCCAGAAGGGGAAC...
8,del,GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACT...,11,NaN,15,0.485338,False,70,23.0,True,NaN,GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACT...,9251,TGCCACCCATCGCGGCTCCCATT,GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACT...,GGGTGCCTACTTGCCACCCATCGCGGCTCCCCTCCAGAAGGGGAAC...
9,del,GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACT...,13,NaN,15,0.454183,False,70,23.0,True,NaN,GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACT...,9251,CCACCCATCGCGGCTCCCCTTTA,GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACT...,GGGTGCCTACTTGCCACCCATCGCGGCTCCCCTCCAGAAGGGGAAC...


In [118]:
complete_df.to_csv('new_maxent_mers.csv', index=False)

Name sequences by id number, pos, and length that way its easier to parse later? need all that information otherwise wont be able to make match and account for predicted frequency  

In [201]:
id_lst = list(complete_df.library_key)
pos_lst = list(complete_df['Genotype position'])
length_lst = list(complete_df.Length)
maxent_lst = list(complete_df.maxent_mer)

In [206]:
with open("maxent_fasta.txt", "w") as ofile:
    for i in range(len(maxent_lst)):
        ofile.write(">" + str(id_lst[i]) + "\t" +str(pos_lst[i]) + "\t" + str(length_lst[i]) +"\n" + maxent_lst[i] +"\n")
    ofile.close()

Create fasta file for insertions and deletions independently

Process MaxEnt values, then append back to genos

Splice likelihood for each id --> multiply every genotype by likelidhood